# Intelligent Query Processing
## Row Mode Memory Grant Feedback
Row Mode Memory Grant feedback is an example of Intelligent Query Processing that can be enabled by using dbcompat = 150. Follow the steps in this notebook to see the performance difference when the query optimizer recognizes a suboptimal memory grant and adapts dynamically.

**Prerequisites**: Restore the WideWorldImportersDW full backup. Then run the **extendwwidw.sql** script to customize the database. If you have already done this, proceed to the next steps

## Step 1: Enable the database for row mode memory grant feedback
Change dbcompat to 150, clear the procedure cache, and ensure the buffer pool has warm cache for OrderHistory to make the query comparison fair.

In [5]:
-- Step 1: Make sure this database is in compatibility level 150 and clear procedure cache for this database. Also bring the table into cache to compare warm cache queries
USE [WideWorldImportersDW]
GO
ALTER DATABASE [WideWorldImportersDW] SET COMPATIBILITY_LEVEL = 150
GO
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE
GO
SELECT COUNT(*) FROM [Fact].[OrderHistory]
GO

Commands completed successfully.

Total execution time: 00:00:00.0011873

Commands completed successfully.

Total execution time: 00:00:00.0036616

Commands completed successfully.

Total execution time: 00:00:00.0038661

(1 row affected)

Total execution time: 00:00:00.2976729

(No column name)
3702592


## Step 2: Simulate statistics out of sync or poor estimation
Let's use a little magic here to simulate that the statistics for the cardinality of the OrderHistory table doesn't match the actual cardinality.

In [6]:
-- Step 2: Simulate statistics out of date
UPDATE STATISTICS Fact.OrderHistory 
WITH ROWCOUNT = 1000
GO

Commands completed successfully.

Total execution time: 00:00:00.0471979

## Step 3: Run a typical data analytics query with joins
Run a query that gets order and stock item data. Force a hash join since it will need a memory grant. Since the optimizer thinks that OrderHistory only has 1000 rows it will under allocate memory for the grant causinmg a spill to disk, thus slowing down the query. Ignore the warning which just says we forced the type of join.

In [7]:
SELECT fo.[Order Key], fo.Description,si.[Lead Time Days]
FROM  Fact.OrderHistory AS fo
INNER HASH JOIN Dimension.[Stock Item] AS si 
ON fo.[Stock Item Key] = si.[Stock Item Key]
WHERE fo.[Lineage Key] = 9
AND si.[Lead Time Days] > 19
GO

Warning: The join order has been enforced because a local join hint is used.

(66416 rows affected)

Total execution time: 00:00:29.6932207

Displaying Top 5000 rows.

Order Key,Description,Lead Time Days
3662222,Tape dispenser (Black),20
3662620,Tape dispenser (Black),20
3664585,Tape dispenser (Black),20
3693219,Tape dispenser (Black),20
3693222,Tape dispenser (Black),20
3693225,Tape dispenser (Black),20
3693230,Tape dispenser (Black),20
3693362,Tape dispenser (Black),20
3693363,Tape dispenser (Black),20
3693364,Tape dispenser (Black),20


## Step 4: Run the query again.
This is the exact same query. This time the optimizer has recognized the grant was too small so will allocate a bigger grant and store it in the cached plan (but this information is not saved to disk). Notice again there are no comments in the cell so the query text is exactly the same. The query now should run signifanctly faster.

In [8]:
SELECT fo.[Order Key], fo.Description,si.[Lead Time Days]
FROM  Fact.OrderHistory AS fo
INNER HASH JOIN Dimension.[Stock Item] AS si 
ON fo.[Stock Item Key] = si.[Stock Item Key]
WHERE fo.[Lineage Key] = 9
AND si.[Lead Time Days] > 19
GO

(66416 rows affected)

Total execution time: 00:00:02.9961746

Displaying Top 5000 rows.

Order Key,Description,Lead Time Days
226141,Air cushion machine (Blue),20
228959,Air cushion machine (Blue),20
202156,Air cushion machine (Blue),20
209868,Air cushion machine (Blue),20
213092,Air cushion machine (Blue),20
204431,Air cushion machine (Blue),20
3702320,Air cushion machine (Blue),20
3702376,Air cushion machine (Blue),20
3700239,Air cushion machine (Blue),20
3700399,Air cushion machine (Blue),20


## Step 5: Restore database to its previous state
Restore the statistics to their correct value and rebuild the index

In [7]:
-- Step 5: Restore table and clustered index back to its original state
UPDATE STATISTICS Fact.OrderHistory 
WITH ROWCOUNT = 3702592
GO
ALTER TABLE [Fact].[OrderHistory] DROP CONSTRAINT [PK_Fact_OrderHistory]
GO
ALTER TABLE [Fact].[OrderHistory] ADD  CONSTRAINT [PK_Fact_OrderHistory] PRIMARY KEY NONCLUSTERED 
(
	[Order Key] ASC,
	[Order Date Key] ASC
)
GO

Commands completed successfully.

Total execution time: 00:00:00.0472133

Commands completed successfully.

Total execution time: 00:00:00.0049750

Commands completed successfully.

Total execution time: 00:00:01.2352462